In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../scripts/')

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 196

In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
%aimport shapeslib

In [5]:
income_file = '../../../ddf--gapminder--fasttrack/ddf--datapoints--mincpcap_cppp--by--country--time.csv'
gini_file =  '../../../ddf--gapminder--fasttrack/ddf--datapoints--gini--by--country--time.csv'

In [14]:
shapes_file = '../wip/smoothshape/ddf--datapoints--population_percentage--by--country--year--coverage_type--bracket.csv'
known_shapes = pd.read_csv(shapes_file)

In [6]:
bracket_number_from_income = shapeslib.bracket_number_from_income

In [7]:
income = pd.read_csv(income_file).set_index(['country', 'time'])
gini = pd.read_csv(gini_file).set_index(['country', 'time'])

income.index.names = ['country', 'year']
gini.index.names = ['country', 'year']

income.columns = ['income']
gini.columns = ['gini']

In [8]:
income_gini = pd.concat([income, gini], axis=1)

In [9]:
income_gini

income   gini
country year               
afg     1800  0.9194  30.50
        1801  0.9194  30.50
        1802  0.9194  30.50
        1803  0.9194  30.50
        1804  0.9194  30.50
...              ...    ...
zwe     2046  5.5473  50.86
        2047  5.6645  50.86
        2048  5.7845  50.86
        2049  5.9075  50.86
        2050  6.0332  50.86

[49447 rows x 2 columns]

In [11]:
income_gini = income_gini.dropna(how='any')

In [12]:
income_gini_noc = income_gini.reset_index(drop=True).drop_duplicates()

In [16]:
known_shapes_list = known_shapes[['country', 'year']].drop_duplicates().values.tolist()

In [18]:
known_shapes_list = [tuple(x) for x in known_shapes_list]

In [20]:
known_income_gini = income_gini.loc[known_shapes_list]

In [21]:
known_income_gini

income   gini
country year               
ago     1981  5.5461  53.97
        1982  5.3525  53.87
        1983  5.3818  53.77
        1984  5.5050  53.67
        1985  5.4991  53.56
...              ...    ...
zwe     2015  4.4835  43.94
        2016  4.1485  44.14
        2017  3.9717  44.34
        2018  4.2896  47.21
        2019  4.0536  50.26

[6389 rows x 2 columns]

In [22]:
known_income_gini.to_csv('../wip/income_gini_for_known_shape_countries.csv')

In [13]:
from multiprocessing import Pool

In [23]:
def get_distances_res(v):
    i = v[0]
    g = v[1]
    cno, neis = shapeslib.get_neighbors(i, g)
    return (i, g), (cno, neis)

with Pool(11) as p:
    res_distances = p.map(get_distances_res, income_gini_noc.values)

In [24]:
res_distances_dict = dict(res_distances)

In [25]:
i, g = shapeslib.get_income_gini(('ago', 1800), income_gini)
res_distances_dict[(i, g)]

(5,
 [('hti', 1994),
  ('hti', 1995),
  ('caf', 2003),
  ('caf', 2001),
  ('caf', 2006),
  ('caf', 2004),
  ('hti', 1993),
  ('hti', 1996),
  ('swz', 1996),
  ('caf', 2005),
  ('hti', 1992),
  ('caf', 2002),
  ('caf', 2000),
  ('caf', 2014),
  ('caf', 2013),
  ('caf', 2015),
  ('hti', 1997),
  ('caf', 2016),
  ('caf', 2017),
  ('hti', 1991),
  ('caf', 2018),
  ('caf', 2019),
  ('caf', 1999),
  ('hti', 1990),
  ('hti', 1989),
  ('swz', 1995),
  ('hti', 1998),
  ('hti', 1988),
  ('caf', 2007),
  ('caf', 2009),
  ('swz', 1997),
  ('zmb', 2015),
  ('caf', 2010),
  ('zmb', 2016),
  ('zmb', 2019),
  ('zmb', 2017),
  ('zmb', 2018),
  ('zmb', 2014),
  ('caf', 2011),
  ('caf', 2008),
  ('hti', 1999),
  ('caf', 1998),
  ('caf', 1997),
  ('zmb', 1992),
  ('caf', 2012),
  ('zmb', 2013),
  ('zmb', 2012),
  ('mwi', 1987),
  ('zmb', 2011),
  ('caf', 1996)])

In [26]:
all_neighbours = []
for cy in income_gini.index.values:
    i, g = shapeslib.get_income_gini(cy, income_gini)
    all_neighbours.append((cy, res_distances_dict[(i, g)]))

In [27]:
all_neighbours = dict(all_neighbours)

In [28]:
all_neighbours[('afg', 1800)]

(10,
 [('npl', 1983),
  ('npl', 1981),
  ('npl', 1982),
  ('tza', 1984),
  ('npl', 1985),
  ('npl', 1984),
  ('tza', 1983),
  ('npl', 1986),
  ('pak', 1981),
  ('npl', 1987),
  ('idn', 1987),
  ('tza', 1985),
  ('idn', 1988),
  ('idn', 1986),
  ('pak', 1982),
  ('chn', 1989),
  ('tza', 1982),
  ('idn', 1989),
  ('rwa', 1985),
  ('chn', 1988),
  ('idn', 1998),
  ('rwa', 1984),
  ('npl', 1988),
  ('eth', 1999),
  ('chn', 1987),
  ('pak', 1983),
  ('rwa', 1986),
  ('eth', 2003),
  ('idn', 1985),
  ('eth', 2000),
  ('rwa', 1981),
  ('rwa', 1982),
  ('idn', 1990),
  ('rwa', 1983),
  ('ind', 1981),
  ('eth', 2002),
  ('tjk', 1999),
  ('eth', 2001),
  ('bdi', 1990),
  ('pak', 1984),
  ('tza', 1981),
  ('ind', 1982),
  ('npl', 1989),
  ('idn', 1991),
  ('tjk', 1998),
  ('bdi', 1989),
  ('idn', 1981),
  ('idn', 1982),
  ('chn', 1990),
  ('chn', 1986)])

In [29]:
import json

In [30]:
all_neighbours_json = dict()

for k, v in all_neighbours.items():
    c, y = k
    if c not in all_neighbours_json.keys():
        all_neighbours_json[c] = dict()
    all_neighbours_json[c][y] = {
        "countries": v[0],
        "neighbours": [list(x) for x in v[1]]
    }

In [31]:
all_neighbours_json['afg'][1900]

{'countries': 8,
 'neighbours': [['bdi', 1996],
  ['bdi', 2000],
  ['bdi', 2019],
  ['bdi', 2001],
  ['bdi', 2018],
  ['bdi', 2017],
  ['bdi', 2016],
  ['bdi', 2015],
  ['ner', 1993],
  ['tcd', 2000],
  ['tza', 2004],
  ['sle', 2004],
  ['tcd', 1999],
  ['tza', 2003],
  ['sle', 2001],
  ['bdi', 2014],
  ['tcd', 2001],
  ['mmr', 1988],
  ['mmr', 1991],
  ['mmr', 1989],
  ['tcd', 1996],
  ['mmr', 1990],
  ['tcd', 1997],
  ['tza', 2005],
  ['mmr', 1992],
  ['mdg', 1997],
  ['mmr', 1981],
  ['mmr', 1987],
  ['mmr', 1993],
  ['tcd', 1998],
  ['mmr', 1982],
  ['mmr', 1983],
  ['mmr', 1986],
  ['bdi', 1997],
  ['mmr', 1994],
  ['mdg', 2005],
  ['mmr', 1984],
  ['mmr', 1985],
  ['mdg', 1998],
  ['sle', 2003],
  ['mmr', 1995],
  ['mwi', 2004],
  ['tcd', 2002],
  ['bdi', 2013],
  ['tcd', 1995],
  ['mmr', 1996],
  ['sle', 2002],
  ['sle', 2000],
  ['mmr', 1997],
  ['tcd', 1993]]}

In [32]:
fp = open('../wip/neighbours_list.json', 'w')
json.dump(all_neighbours_json, fp, indent=2)